# Notebook 1 - Expertensysteme


## 1) Lernziele
- Du modellierst eine kleine Wissensbasis mit Fakten und Regeln.
- Du verstehst Forward Chaining als transparente Inferenzstrategie.
- Du vergleichst regelbasiert, fallbasiert und Entscheidungsbaum auf derselben Aufgabe.
- Du erkennst Grenzen bei Konflikten, Unsicherheit und Wartbarkeit.


## 2) Warm-up Spielzelle: Regelbasiertes Mini-Expertensystem
- Waehle beobachtete Symptome.
- Starte die Diagnose.
- Lies aktive Regeln, neue Fakten, Schlussfolgerung und Begruendungskette.


In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown

# Basisfakten, die Lernende direkt setzen koennen.
SYMPTOMS = ["fieber", "husten", "kopfschmerz", "ausschlag", "muedigkeit", "halsschmerz"]

# Kleine Wissensbasis (Regeln) fuer dieselbe Aufgabe in allen drei Ansaetzen.
RULES = [
    {"name": "R1", "if": {"fieber", "husten"}, "then": "verdacht_grippaler_infekt", "priority": 2, "confidence": 0.85},
    {"name": "R2", "if": {"fieber", "kopfschmerz"}, "then": "verdacht_virusinfekt", "priority": 1, "confidence": 0.70},
    {"name": "R3", "if": {"ausschlag", "fieber"}, "then": "abklaerung_exanthem", "priority": 3, "confidence": 0.90},
    {"name": "R4", "if": {"husten", "halsschmerz"}, "then": "verdacht_atemwegsreizung", "priority": 1, "confidence": 0.60},
    {"name": "R5", "if": {"muedigkeit", "kopfschmerz"}, "then": "verdacht_stressreaktion", "priority": 1, "confidence": 0.55},
]

GOAL_ORDER = [
    "abklaerung_exanthem",
    "verdacht_grippaler_infekt",
    "verdacht_virusinfekt",
    "verdacht_atemwegsreizung",
    "verdacht_stressreaktion",
]


def forward_chain_with_trace(initial_facts, rules, confidence_threshold=0.0, use_priority=False):
    """Forward Chaining mit optionaler Priorisierung und Confidence-Schwelle."""
    known = set(initial_facts)
    trace = []

    ordered_rules = sorted(rules, key=lambda r: r["priority"], reverse=True) if use_priority else list(rules)

    changed = True
    while changed:
        changed = False
        for r in ordered_rules:
            if r["confidence"] < confidence_threshold:
                continue
            if r["if"].issubset(known) and r["then"] not in known:
                known.add(r["then"])
                trace.append({
                    "rule": r["name"],
                    "premises": sorted(r["if"]),
                    "new_fact": r["then"],
                    "confidence": r["confidence"],
                })
                changed = True
    return known, trace


def choose_conclusion(known):
    for g in GOAL_ORDER:
        if g in known:
            return g
    return "kein_klarer_verdacht"


symptom_boxes = {s: widgets.Checkbox(value=False, description=s) for s in SYMPTOMS}
run_btn = widgets.Button(description="Diagnose", button_style="info")
out = widgets.Output()


def on_diagnose(_):
    selected = {s for s, cb in symptom_boxes.items() if cb.value}
    known, trace = forward_chain_with_trace(selected, RULES)
    new_facts = sorted(known - selected)
    conclusion = choose_conclusion(known)

    with out:
        out.clear_output()
        display(Markdown(f"**Ausgangsfakten:** `{sorted(selected)}`"))

        if trace:
            active_rules = [t["rule"] for t in trace]
            display(Markdown(f"**Aktive Regeln:** `{active_rules}`"))
            display(Markdown(f"**Neue Fakten:** `{new_facts}`"))
            chain_lines = ["**Begruendungskette:**"]
            for i, t in enumerate(trace, 1):
                chain_lines.append(
                    f"- Schritt {i}: {t['rule']} feuert, weil {t['premises']} wahr sind -> {t['new_fact']}"
                )
            display(Markdown("\n".join(chain_lines)))
        else:
            display(Markdown("**Aktive Regeln:** `[]`"))
            display(Markdown("**Neue Fakten:** `[]`"))
            display(Markdown("**Begruendungskette:** keine Regel hat gegriffen."))

        display(Markdown(f"**Schlussfolgerung:** `{conclusion}`"))


run_btn.on_click(on_diagnose)

display(widgets.VBox([
    widgets.HTML("<b>Symptome waehlen</b>"),
    widgets.HBox(list(symptom_boxes.values())),
    run_btn,
    out,
]))


### Wissensbasis, Inferenzmaschine, Benutzeroberflaeche
- Wissensbasis: Fakten und Wenn-Dann-Regeln mit Prioritaet und Confidence.
- Inferenzmaschine: Forward Chaining leitet neue Fakten aus bekannten Fakten ab.
- Benutzeroberflaeche: Checkboxen und Diagnose-Button machen den Schlussprozess sichtbar.
- Erklaerbarkeit: Jede abgeleitete Aussage bleibt ueber die Regelspur nachvollziehbar.


## 3) Minimalmodell und Pseudocode (Forward Chaining)
- Start mit bekannten Fakten.
- Pruefe wiederholt alle Regeln.
- Leite neue Fakten ab, bis keine Regel mehr neue Information liefert.

```text
INPUT facts, rules
known <- facts
trace <- []
REPEAT
  changed <- False
  FOR r IN rules
    IF r.if subset of known AND r.then not in known
      known <- known U {r.then}
      trace <- trace + (r.name, r.if, r.then)
      changed <- True
UNTIL changed = False
RETURN known, trace
```


In [ ]:
# Kleine, direkte Referenzimplementierung des Pseudocodes.
def fc_minimal(facts, rules):
    known = set(facts)
    trace = []
    changed = True
    while changed:
        changed = False
        for name, premise, conclusion in rules:
            if premise.issubset(known) and conclusion not in known:
                known.add(conclusion)
                trace.append((name, sorted(premise), conclusion))
                changed = True
    return known, trace

minimal_facts = {"fieber", "husten"}
minimal_rules = [
    ("M1", {"fieber", "husten"}, "verdacht_grippaler_infekt"),
    ("M2", {"verdacht_grippaler_infekt"}, "ruhe_empfohlen"),
]

known_demo, trace_demo = fc_minimal(minimal_facts, minimal_rules)
print("Known:", sorted(known_demo))
print("Trace:", trace_demo)


## 4) Drei Ansaetze auf derselben Aufgabe
- Aufgabe: Symptome -> Verdacht.
- Gleiche Eingabeidee in allen drei Ansaetzen.
- Unterschied liegt in der Wissensreprasentation und Ableitungslogik.


### 4a) Regelbasiert


In [ ]:
# Regelbasierte Vorhersage nutzt dieselbe RULES-Liste wie das Warm-up.
def predict_rule_based(symptoms):
    known, trace = forward_chain_with_trace(symptoms, RULES)
    return choose_conclusion(known), trace

example_symptoms = {"fieber", "husten", "muedigkeit"}
rule_pred, rule_trace = predict_rule_based(example_symptoms)
print("Symptome:", sorted(example_symptoms))
print("Regelbasiertes Ergebnis:", rule_pred)
print("Regelspur:", [t["rule"] for t in rule_trace])


### 4b) Fallbasiert (10 Faelle, Similarity als sortierte Tabelle)
- Similarity: Jaccard = Schnittmenge / Vereinigung.
- Hoehere Werte bedeuten aehnlichere Symptomkonstellationen.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

CASE_LIBRARY = [
    {"id": "F1", "symptoms": {"fieber", "husten"}, "diagnosis": "verdacht_grippaler_infekt"},
    {"id": "F2", "symptoms": {"fieber", "kopfschmerz"}, "diagnosis": "verdacht_virusinfekt"},
    {"id": "F3", "symptoms": {"ausschlag", "fieber"}, "diagnosis": "abklaerung_exanthem"},
    {"id": "F4", "symptoms": {"husten", "halsschmerz"}, "diagnosis": "verdacht_atemwegsreizung"},
    {"id": "F5", "symptoms": {"muedigkeit", "kopfschmerz"}, "diagnosis": "verdacht_stressreaktion"},
    {"id": "F6", "symptoms": {"fieber", "husten", "kopfschmerz"}, "diagnosis": "verdacht_grippaler_infekt"},
    {"id": "F7", "symptoms": {"husten"}, "diagnosis": "verdacht_atemwegsreizung"},
    {"id": "F8", "symptoms": {"ausschlag", "kopfschmerz"}, "diagnosis": "abklaerung_exanthem"},
    {"id": "F9", "symptoms": {"muedigkeit", "halsschmerz"}, "diagnosis": "verdacht_virusinfekt"},
    {"id": "F10", "symptoms": {"fieber", "muedigkeit"}, "diagnosis": "verdacht_virusinfekt"},
]


def jaccard(a, b):
    u = len(a | b)
    return (len(a & b) / u) if u else 0.0


def case_based_table(query_symptoms, cases):
    rows = []
    for c in cases:
        common = sorted(query_symptoms & c["symptoms"])
        rows.append({
            "case_id": c["id"],
            "diagnosis": c["diagnosis"],
            "symptoms": ", ".join(sorted(c["symptoms"])),
            "matches": ", ".join(common),
            "similarity": round(jaccard(query_symptoms, c["symptoms"]), 3),
        })
    return pd.DataFrame(rows).sort_values("similarity", ascending=False).reset_index(drop=True)

query = {"fieber", "husten", "kopfschmerz"}
sim_df = case_based_table(query, CASE_LIBRARY)
print("Query:", sorted(query))
display(sim_df)


### 4c) Entscheidungsbaum (if-elif)


In [ ]:
# Kleiner, expliziter Entscheidungsbaum fuer dieselbe Aufgabe.
def predict_tree(symptoms):
    path = []
    if "fieber" in symptoms:
        path.append("fieber = ja")
        if "ausschlag" in symptoms:
            path.append("ausschlag = ja")
            return "abklaerung_exanthem", path
        path.append("ausschlag = nein")
        if "husten" in symptoms:
            path.append("husten = ja")
            return "verdacht_grippaler_infekt", path
        path.append("husten = nein")
        return "verdacht_virusinfekt", path
    path.append("fieber = nein")
    if "husten" in symptoms and "halsschmerz" in symptoms:
        path.append("husten = ja und halsschmerz = ja")
        return "verdacht_atemwegsreizung", path
    path.append("husten/halsschmerz nicht gemeinsam")
    return "verdacht_stressreaktion", path

example_tree = {"fieber", "husten", "kopfschmerz"}
tree_pred, tree_path = predict_tree(example_tree)
print("Symptome:", sorted(example_tree))
print("Baum-Ergebnis:", tree_pred)
print("Pfad:")
for step in tree_path:
    print("-", step)


## 5) Datensatz Einblick zur Fallbibliothek
- Die Fallbibliothek enthaelt 10 kleine, manuell gepflegte Faelle.
- Visualisierung: Haeufigkeit der Symptome in der Bibliothek (matplotlib).


In [ ]:
# Ein einfacher Einblick: Wie oft kommt jedes Symptom in den 10 Faellen vor?
all_symptoms = sorted({s for c in CASE_LIBRARY for s in c["symptoms"]})
counts = {s: sum(1 for c in CASE_LIBRARY if s in c["symptoms"]) for s in all_symptoms}

plt.figure(figsize=(7, 3.5))
plt.bar(list(counts.keys()), list(counts.values()), color='tab:blue')
plt.title('Symptomhaeufigkeit in der Fallbibliothek')
plt.ylabel('Anzahl Faelle')
plt.xticks(rotation=25)
plt.tight_layout()
plt.show()


## 6) Aufsteigende Erweiterungen
1. Konfliktloesung: Regelprioritaet steuert, welche Regel zuerst feuert.
2. Unsicherheit: Confidence pro Regel plus Schwellwert fuer das Feuern.
3. Wartbarkeit: Grenzen bei Wissensakquise, Inkonsistenzen und Skalierung.


In [ ]:
# Erweiterung 1+2: Vergleich ohne/mit Prioritaet und mit Confidence-Schwelle.
scenario = {"fieber", "husten", "kopfschmerz", "ausschlag"}

known_a, trace_a = forward_chain_with_trace(scenario, RULES, confidence_threshold=0.0, use_priority=False)
known_b, trace_b = forward_chain_with_trace(scenario, RULES, confidence_threshold=0.75, use_priority=True)

print("Szenario:", sorted(scenario))
print("\nOhne Prioritaet, ohne Schwelle")
print("- aktive Regeln:", [t["rule"] for t in trace_a])
print("- Schlussfolgerung:", choose_conclusion(known_a))

print("\nMit Prioritaet, Confidence >= 0.75")
print("- aktive Regeln:", [t["rule"] for t in trace_b])
print("- Schlussfolgerung:", choose_conclusion(known_b))


### Wartbarkeit: kurze Grenzen
- Wissensakquise: Regeln muessen explizit erhoben und abgestimmt werden.
- Skalierung: Viele Regeln erhoehen Konflikte, Redundanzen und Testaufwand.
- Pflege: Jede neue Ausnahme kann unerwartete Seiteneffekte auf alte Regeln haben.
- Betrieb: Regelversionierung und Regressionstests sind fuer Stabilitaet zentral.


## 7) Mini Leitfaden (7 bis 10 Minuten)
- Minute 0-1: Lernziele lesen und Fakten-Regel-Idee aktivieren.
- Minute 1-3: Warm-up mit zwei Symptomkombinationen testen.
- Minute 3-5: Regelspur mit Pseudocode abgleichen.
- Minute 5-7: Fallbasierten Similarity-Table lesen und Top-Fall pruefen.
- Minute 7-9: Entscheidungsbaum-Pfad mit Regelspur vergleichen.
- Minute 9-10: Erweiterungen (Prioritaet/Confidence/Wartbarkeit) reflektieren.


## 8) Mini Uebungen
1. Ergaenze eine neue Regel fuer `fieber + ausschlag + kopfschmerz` und teste die Regelspur.
2. Setze die Confidence-Schwelle auf 0.8 und dokumentiere, welche Regeln nicht mehr feuern.
3. Vergleiche fuer dieselbe Symptommenge die Ergebnisse von Regelbasis, Fallbasis und Baum.
4. Finde einen Fall, bei dem der Baum eine andere Schlussfolgerung liefert als CBR, und begruende warum.
5. Beschreibe in 3 Stichpunkten, wie du die Wissensbasis wartbarer machen wuerdest.
